In [4]:
import codenamesLLM
import pandas as pd
import openpyxl
from tqdm import tqdm

c:\Users\miche\OneDrive\Desktop\Codenames Progetto PDE\Codenames-LLM\codenames_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Path to your Excel file
file_path = "experiment_data\model_tournament\model_tournament_mich1.xlsx" #compy the model_tournament_input to use one new

# Read the Excel file into a DataFrame
df = pd.read_excel(file_path)

# Iterate through the rows using titled columns
for index, row in tqdm(df.iterrows(), total=len(df)):
    # Access values using column titles
    red_team = row['red_model']  # Replace with the actual column title for the red team
    blue_team = row['blue_model']  # Replace with the actual column title for the blue team
    playable = True

    already_played = pd.notna(row['winner'])
    
    if ((pd.notna(red_team) and pd.notna(blue_team)) and not(already_played)):  # Check if both values are not NaN
        try:
            print(f"Playing {red_team} vs {blue_team}...")
            # Call your function and get the result
            result = codenamesLLM.play_game(red_model = red_team, blue_model = blue_team)
            print(result)
            red_stats = codenamesLLM.analyze_team_guesses(result[3], "RED")
            blue_stats = codenamesLLM.analyze_team_guesses(result[3], "BLUE")

            df.at[index, 'red_model'] = red_team
            df.at[index, 'red_model'] = red_team
            df.at[index, 'winner'] = result[0]  
            df.at[index, 'red_avg_words_2guess'] = red_stats['average_expected_guesses']
            df.at[index, 'blue_avg_words_2guess'] = blue_stats['average_expected_guesses']
            df.at[index, 'red_avg_words_guessed'] = red_stats['average_correct_guesses']
            df.at[index, 'blue_avg_words_guessed'] = blue_stats['average_correct_guesses']
            df.at[index, 'reason'] = result[1]
            df.at[index, 'red_turns'] = red_stats['total_hints']
            df.at[index, 'blue_turns'] = blue_stats['total_hints']
            df.at[index, 'red_cib'] = result[4]
            df.at[index, 'blue_cib'] = result[5]

        except Exception as e:
            print(f"skipped game between {red_team} and {blue_team}: {e}")

    # Write the updated DataFrame back to the same Excel file
    df.to_excel(file_path, index=False)
    if not already_played:
        print(f"Executed {red_team} vs {blue_team}")


100%|██████████| 90/90 [00:03<00:00, 23.20it/s]


In [ ]:
def generate_model_statistics(input_file, output_file):
    # Load the data
    df = pd.ExcelFile(input_file).parse(0)  # Assumes the data is in the first sheet

    # Unique models
    models = pd.concat([df['red_model'], df['blue_model']]).unique()

    # Prepare the statistics dictionary
    stats = {
        'model_name': [],
        'games_played': [],
        'wins': [],
        'wins_as_red': [],
        'wins_as_blue': [],
        'wins_by_cards_finished': [],
        'wins_by_killer_word': [],
        'killer_word_losses': [],
        'avg_words_to_guess': [],
        'avg_words_guessed': [],
        'avg_rounds': [],
        'avg_rounds_when_cards_finished': [],
        'avg_words_to_guess_when_win': [],
        'avg_words_to_guess_when_lose': [],
        'avg_words_guessed_when_win': [],
        'avg_words_guessed_when_lose': [],
    }

    for model in models:
        # Filter games where the model is playing
        red_games = df[df['red_model'] == model]
        blue_games = df[df['blue_model'] == model]
        all_games = pd.concat([red_games, blue_games])

        # Wins
        wins_as_red = (red_games['winner'] == 'RED').sum()
        wins_as_blue = (blue_games['winner'] == 'BLUE').sum()
        total_wins = wins_as_red + wins_as_blue

        # Losses and reasons
        killer_word_losses = all_games[(all_games['reason'] == 'killer word selected') & 
                                       ((all_games['red_model'] == model) & (all_games['winner'] == 'BLUE') |
                                        (all_games['blue_model'] == model) & (all_games['winner'] == 'RED'))].shape[0]

        # Wins by reason
        wins_by_cards_finished = all_games[(all_games['reason'] == 'cards finished') & 
                                            ((all_games['red_model'] == model) & (all_games['winner'] == 'RED') |
                                             (all_games['blue_model'] == model) & (all_games['winner'] == 'BLUE'))].shape[0]
        wins_by_killer_word = total_wins - wins_by_cards_finished

        # Games played
        games_played = all_games.shape[0]

        # Average statistics
        avg_words_to_guess = all_games[['red_avg_words_2guess', 'blue_avg_words_2guess']].mean().mean()
        avg_words_guessed = all_games[['red_avg_words_guessed', 'blue_avg_words_guessed']].mean().mean()
        avg_rounds = all_games[['red_turns', 'blue_turns']].mean().mean()
        avg_rounds_when_cards_finished = all_games[all_games['reason'] == 'cards finished'][['red_turns', 'blue_turns']].mean().mean()

        # Words to guess when win or lose
        avg_words_to_guess_when_win = all_games[(all_games['winner'] == 'RED') & (all_games['red_model'] == model) |
                                                (all_games['winner'] == 'BLUE') & (all_games['blue_model'] == model)][
            ['red_avg_words_2guess', 'blue_avg_words_2guess']].mean().mean()
        avg_words_to_guess_when_lose = all_games[(all_games['winner'] == 'BLUE') & (all_games['red_model'] == model) |
                                                 (all_games['winner'] == 'RED') & (all_games['blue_model'] == model)][
            ['red_avg_words_2guess', 'blue_avg_words_2guess']].mean().mean()

        avg_words_guessed_when_win = all_games[(all_games['winner'] == 'RED') & (all_games['red_model'] == model) |
                                               (all_games['winner'] == 'BLUE') & (all_games['blue_model'] == model)][
            ['red_avg_words_guessed', 'blue_avg_words_guessed']].mean().mean()
        avg_words_guessed_when_lose = all_games[(all_games['winner'] == 'BLUE') & (all_games['red_model'] == model) |
                                                (all_games['winner'] == 'RED') & (all_games['blue_model'] == model)][
            ['red_avg_words_guessed', 'blue_avg_words_guessed']].mean().mean()

        # Append stats for the model
        stats['model_name'].append(model)
        stats['games_played'].append(games_played)
        stats['wins'].append(total_wins)
        stats['wins_as_red'].append(wins_as_red)
        stats['wins_as_blue'].append(wins_as_blue)
        stats['wins_by_cards_finished'].append(wins_by_cards_finished)
        stats['wins_by_killer_word'].append(wins_by_killer_word)
        stats['killer_word_losses'].append(killer_word_losses)
        stats['avg_words_to_guess'].append(avg_words_to_guess)
        stats['avg_words_guessed'].append(avg_words_guessed)
        stats['avg_rounds'].append(avg_rounds)
        stats['avg_rounds_when_cards_finished'].append(avg_rounds_when_cards_finished)
        stats['avg_words_to_guess_when_win'].append(avg_words_to_guess_when_win)
        stats['avg_words_to_guess_when_lose'].append(avg_words_to_guess_when_lose)
        stats['avg_words_guessed_when_win'].append(avg_words_guessed_when_win)
        stats['avg_words_guessed_when_lose'].append(avg_words_guessed_when_lose)

    # Create a DataFrame from the stats dictionary
    stats_df = pd.DataFrame(stats)

    # Save the resulting DataFrame to an Excel file
    stats_df.to_excel(output_file, index=False)
    print(f"Statistics saved to {output_file}")

In [8]:
generate_model_statistics("experiment_data\model_tournament\model_tournament_mich1.xlsx", "experiment_data\model_tournament\model_tournament_stats_partial.xlsx")

Statistics saved to experiment_data\model_tournament\model_tournament_stats_partial.xlsx
